In [1]:
import os
import yaml

from espnet2.text.build_tokenizer import build_tokenizer
from espnet2.text.token_id_converter import TokenIDConverter

import jiwer

In [2]:
model_root_dir = "/home/pb_deployment/espnet/asr_inference/model_files/e_branchformer_librispeech/"
asr_config_path = os.path.join(model_root_dir, "exp/asr_train_asr_e_branchformer_raw_en_bpe5000_sp/config.yaml")
bpe_model_path = os.path.join(model_root_dir,"data/en_token_list/bpe_unigram5000/bpe.model")

In [3]:
class TruCLeS_Tokenizer:
    def __init__(self, asr_config_path, bpe_model_path):
        self.asr_config_path = asr_config_path
        self.bpe_model_path = bpe_model_path
        self.tokenizer = build_tokenizer(
            token_type='bpe',
            bpemodel=bpe_model_path  
        )
        with open(self.asr_config_path, 'r') as file:
            config_data = yaml.safe_load(file)
        self.tokens_list = config_data.get('token_list', [])
        self.tokenIDConvertor = TokenIDConverter(token_list = self.tokens_list)

        self.ids = dict(zip(self.tokens_list, [i for i in range(len(self.tokens_list))]))

    def text2ids(self, text):
        tokenized = self.tokenizer.text2tokens(text)
        ids = self.tokenIDConvertor.tokens2ids(tokenized)
        return ids
    
    def ids2text(self, ids):
        tokenized = self.tokenIDConvertor.ids2tokens(ids)
        text = self.tokenizer.tokens2text(tokenized)
        return text

    def text2tokens(self, text):
        return self.tokenizer.text2tokens(text)
        
    def tokens2text(self, tokens):
        return self.tokenizer.tokens2text(tokens)

    def tokens2ids(self, tokens):
        return self.tokenIDConvertor.tokens2ids(tokens)

    def ids2tokens(self, ids):
        return self.tokenIDConvertor.ids2tokens(ids)

In [4]:
tokenizer = TruCLeS_Tokenizer(asr_config_path, bpe_model_path)

In [73]:
ref = "HELLO THIS IS THISHYAN RAJ" 
hyp = "HELLO THISH IS THIYAN"
print(jiwer.visualize_alignment(jiwer.process_words(ref, hyp), skip_correct=False, show_measures=False))

def getAlignedHyp(ref, hyp, tokenizer = tokenizer):
    wOut = jiwer.process_words(ref, hyp)
    wRefList = wOut.references[0]
    wHypList = wOut.hypotheses[0]
    wAligns = wOut.alignments[0]
    tokenizedWRefList = tokenizer.text2tokens(wRefList)
    tokenizedWHypList = tokenizer.text2tokens(wHypList)
    
    align_list = []
    for wAlign in wAligns:
        if wAlign.type == "equal":
            align_list = align_list + tokenizedWHypList[wAlign.hyp_start_idx:wAlign.hyp_end_idx]
        elif wAlign.type == "insert":
            align_list.append([])
        elif wAlign.type == "substitute":
            hypWords = tokenizedWHypList[wAlign.hyp_start_idx:wAlign.hyp_end_idx]
            refWords = tokenizedWRefList[wAlign.ref_start_idx:wAlign.ref_end_idx]
            word_list = []
            for hypWord, refWord in zip(hypWords, refWords):
                tOut = jiwer.process_words(" ".join(refWord), " ".join(hypWord))
                tRefList = tOut.references[0]
                tHypList = tOut.hypotheses[0]
                tAligns = tOut.alignments[0]
                for tAlign in tAligns:
                    if tAlign.type == "equal":
                        word_list = word_list + tHypList[tAlign.hyp_start_idx:tAlign.hyp_end_idx]
                    elif tAlign.type == "substitute":
                        word_list = word_list + tRefList[tAlign.ref_start_idx:tAlign.ref_end_idx]
                    elif tAlign.type == "insert":
                        word_list.append("")
            align_list = align_list + [word_list]
    return align_list

align_list = getAlignedHyp(ref, hyp)
print(align_list)

sentence 1
REF: HELLO  THIS IS THISHYAN RAJ
HYP: HELLO THISH IS   THIYAN ***
               S           S   D

[['▁HE', 'LL', 'O'], ['▁THIS', ''], ['▁IS'], ['', '▁THIS', 'HY', 'AN']]


In [76]:
i = 0
for word in align_list:
    if len(word) != 0:
        for tok in word:
            if tok!="":
                print(i, 1)
            else:
                print(i, 0)
            i += 1
    else:
        print(i, 0)
        i += 1

0 1
1 1
2 1
3 1
4 0
5 1
6 0
7 1
8 1
9 1
